# Visuliazation of some paramaters of robot
- this jupyter notbook show the variables related to the robot


## import

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import actionlib
import actionlib.msg
import collections
import math

rospy.init_node('visualized_node')

## display the movemennt of the robot

- an xy graph showing the robot’s position and the orientation 

In [7]:
# First set up the figure and the axis 
fig, ax = plt.subplots()

ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Robot positon")

line, = ax.plot([], [], 'ro')
x_pos=0
y_pos=0
x_data=collections.deque(maxlen=300)
y_data=collections.deque(maxlen=300)
yaw=0
theta=collections.deque(maxlen=300)
arrow = ax.quiver(1,1,1,1, alpha = 0)


def init():
    line.set_data([], [])
    return (line,)


def odom_callback(msg):
    global x_data,y_data,yaw,theta,actual_linear_vel,actual_angular_vel
    x_pos=msg.pose.pose.position.x
    y_pos=msg.pose.pose.position.y
    y_data.append(y_pos)
    x_data.append(x_pos)

    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    theta.append(yaw)
    
def animate(i):
    global x_data,y_data,yaw,arrow,theta
    arrow.remove()
    x = x_data[-1]
    y = y_data[-1]
    line.set_data(x_data, y_data)
    a = np.cos(theta[-1]) 
    b = np.sin(theta[-1])
    arrow = ax.quiver(x, y, a, b,scale = 0.5, color='r', pivot = 'tail', units='xy', alpha=0.6, edgecolor = 'black')
    return (line,arrow)


jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True)

ERROR:tornado.application:Exception in callback <bound method TimerBase._on_timer of <matplotlib.backends.backend_webagg_core.TimerTornado object at 0x7fdad5c98b20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/usr/lib/python3/dist-packages/matplotlib/backend_bases.py", line 1194, in _on_timer
    ret = func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1447, in _step
    still_going = Animation._step(self, *args)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1173, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1192, in _draw_next_frame
    self._draw_frame(framedata)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1755, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipy

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


## compare the actual velocity with cmd_vel

In [5]:
print(x_data)

[-3.397972538200226, -3.398298397028391, -3.39863646774669, -3.3989055522067484, -3.3991931606746126, -3.3994485435789947, -3.399725695250403, -3.400028964863522, -3.4002681407858883, -3.4006563349175725, -3.4008996807004315, -3.401037799754544, -3.4012373844776445, -3.401518601287138, -3.401791874459437, -3.4020178810331227, -3.4020911607127586, -3.4022539535772562, -3.402435554644873, -3.4024833900569544, -3.4026124428998443, -3.4029453979921325, -3.403267427928375, -3.403578740071346, -3.403879285827117, -3.4041690207261954, -3.4044479013005793, -3.4047158839431337, -3.404972923800817, -3.405216320671898, -3.4054476494077472, -3.405667287062112, -3.4058752771375387, -3.4060716032844245, -3.4062599900806547, -3.4064372348811087, -3.4064509471585467, -3.4063604225681834, -3.4062785028119675, -3.406167387761093, -3.406183646865607, -3.4063021649475376, -3.406408492189078, -3.406502656153854, -3.406584549450676, -3.406654060890513, -3.4066662252151105, -3.4064426203680305, -3.4061376765

- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity)

In [4]:
fig1, ax1 = plt.subplots()

ax1.set_xlim(( 0, 100))
ax1.set_ylim((-1.2, 1.2))

linear_cmd_vel=[]
angular_cmd_vel=[]
actual_linear_vel=[]
actual_angular_vel=[]

time=[]
odom_time=[]

line1, = ax1.plot([], [], 'r-')
line2, = ax1.plot([], [], 'b-')
line3, = ax1.plot([], [], 'c-')
line4, = ax1.plot([], [], 'y-')
zero_time = rospy.get_time()

def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    return (line1,line2,line3,line4)

def twist_callback(msg):
    global linear_cmd_vel,angular_cmd_vel,zero_time,time
    linear_cmd_vel.append(msg.linear.x)
    angular_cmd_vel.append(msg.angular.z)
    time.append(rospy.get_time()-zero_time)

def odom_vel_callback(msg):
    global actual_linear_vel,actual_angular_vel,odom_time,zero_time
    actual_linear_vel.append(msg.twist.twist.linear.x)
    actual_angular_vel.append(-1.85*msg.twist.twist.angular.z)
    odom_time.append(rospy.get_time()-zero_time)

def animate1(i):
    global linear_cmd_vel,angular_cmd_vel,time,odom_time,actual_linear_vel,actual_angular_vel
    line1.set_data(time,linear_cmd_vel)
    line2.set_data(time, angular_cmd_vel)
    line3.set_data(odom_time, actual_linear_vel)
    line4.set_data(odom_time, actual_angular_vel)

    return (line1,line2,line3,line4)

jr.subscribe('/odom', Odometry, odom_vel_callback)
jr.subscribe('/cmd_vel', Twist, twist_callback)

anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


- a boxplot displaying the number of reached targets and cancelled targets


## display the robot status
- a boxplot displaying the number of reached targets and cancelled targets
- subscribe /go_to_point/result and update the number of goal and cancel

In [5]:
ngoal=0
ncancel=0
fig2, ax2 = plt.subplots()

ax2.set_xlim(( 0, 2))
ax2.set_ylim((0.0, 10))

def result_callback(msg):
    global ngoal, ncancel
    if msg.result.reached==True:
        ngoal=ngoal+1
        print(ngoal)
    #elif msg.result.reached==False:
    else:
        ncancel=ncancel+1

def animate2(i):
    global ngoal,ncancel
    index=1
    ax2.bar(index,ngoal,label="the number of goal", align="center", color='blue', width=0.35)
    ax2.bar(index+0.35, ncancel, align='center', width=0.35,color='r', label='the number of cancel') 
        
jr.subscribe('/go_to_point/result', rt2_assignment1.msg.MotionActionResult, result_callback)

anim = animation.FuncAnimation(fig2, animate2, 
                               frames=100, interval=20,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## show the required time to reach goals
- a hist plot showing the time required to reach targets
- prepare the topic /reached_time

In [6]:
fig3, ax3 = plt.subplots()
ax3.set_ylim((0.0, 5))
goal_time_array=[]
index=0
def init3():
    print('init')
def reached_callback(msg):
    global goal_time_array
    goal_time_array.append(msg.data)
    print(goal_time_array)
    
def animate3(i):
    global goal_time_array
    ax3.hist(goal_time_array,bins=10, align='mid',ec="green", fc="green", alpha=0.5) 


jr.subscribe('/reached_time', Float32 , reached_callback)

anim = animation.FuncAnimation(fig3, animate3,init_func=init3, 
                               frames=100, interval=200,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …